In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [3]:
!mkdir -p data results logs


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [6]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


Run following if the library code is modified locally

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
#            'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
#            'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
#            'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
#            'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
#            'MAXR', 'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
#            'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
#            'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM', 'RBLX',
#            'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM', 'STNE','SWAV',
#            'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
#            'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
model = "gstock-4a-test"
tickers = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)


In [9]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,ADYEY,6.78,0.95,149.44,139.38,15.88,14.230000,-0.10,0.75,144.72,126.36,16.469999,0.04,0.62,99.02,103.27,13.160000,-0.17,9.000000
49,PROSF,43.97,0.93,494.03,524.74,71.27,81.980003,0.15,0.82,572.37,440.08,75.040001,0.05,0.70,200.33,339.10,71.019997,-0.00,8.833333
13,CPG,0.75,0.89,61.81,47.19,7.78,5.590000,-0.28,0.73,49.92,47.00,7.550000,-0.03,0.65,25.80,45.59,6.010000,-0.23,8.507143
10,COTY,2.89,0.91,66.40,89.76,12.26,10.630000,-0.13,0.68,62.64,66.88,12.540000,0.02,0.68,57.66,41.58,11.320000,-0.08,8.407692
22,ERIC,5.44,0.92,45.48,64.33,5.96,6.540000,0.10,0.68,72.99,95.87,6.210000,0.04,0.61,14.16,43.59,5.940000,-0.00,8.400000
67,VWAPY,7.45,0.94,93.59,109.86,13.98,14.880000,0.06,0.68,357.53,369.02,14.610000,0.04,0.13,-266.92,41.32,13.360000,-0.04,8.400000
58,STLA,5.26,0.91,109.17,68.53,18.86,16.700001,-0.11,0.78,270.16,229.70,18.290001,-0.03,0.29,-62.24,29.82,17.990000,-0.05,8.281818
0,ABB,13.46,0.94,158.98,113.33,35.14,33.529999,-0.05,0.80,334.21,278.32,34.119999,-0.03,0.32,-41.43,39.03,33.419998,-0.05,8.200000
54,ORAN,8.50,0.94,45.06,37.04,12.48,11.810000,-0.05,0.74,195.10,189.10,12.900000,0.03,0.12,-131.95,6.93,12.410000,-0.01,8.200000
56,SE,29.88,0.85,1492.62,1388.34,83.52,121.070000,0.45,0.75,1858.50,1542.15,98.699997,0.18,0.64,1417.02,1078.30,80.040001,-0.04,8.166667


In [ ]:
df1  = pipeline.runModelCombinedVolaR(tickers, model, mod, False, loss=lossfn)

In [11]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False).head(50)


,,Name,Error,Accu,Buy,Sell,Total,Last,Pred,Gain,Alloc
Ticker,Target,,,,,,,,,,
PROSF,adjclose,RROnly,43.62,0.97,537.58,466.35,1003.94,71.27,81.980003,0.15,9.500000
ADYEY,adjclose,RROnly,6.69,0.96,158.37,129.34,287.71,15.88,14.230000,-0.10,9.200000
COTY,adjclose,RROnly,2.84,0.94,83.03,89.97,173.01,12.26,10.630000,-0.13,8.938462
NOK,adjclose,RROnly,2.47,0.94,27.42,36.66,64.09,4.90,5.450000,0.11,8.854545
STLA,adjclose,RROnly,5.24,0.94,111.36,84.21,195.57,18.86,16.700001,-0.11,8.854545
TKAMY,adjclose,RROnly,3.99,0.94,82.39,90.40,172.78,7.83,8.600000,0.10,8.800000
NU,adjclose,RROnly,3.47,0.93,14.38,26.09,40.47,4.66,5.220000,0.12,8.716667
CPG,adjclose,RROnly,0.72,0.90,62.35,36.14,98.49,7.78,5.590000,-0.28,8.642857
COVTY,adjclose,RROnly,11.97,0.92,125.21,160.04,285.25,20.17,22.510000,0.12,8.533333


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [12]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 

sending incremental file list
gstock-4a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
gstock-4a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,640,220 bytes  received 73 bytes  6,426,862.00 bytes/sec
total size is 9,637,296  speedup is 1.00
